In [56]:
# %load src/XORV2.py
'''''''''''''''''''''''''''''''''''''''''''''''''''''
   file name: XORV2.py
   create time: Sun 16 Jul 2017 07:00:20 AM EDT
   author: Jipeng Huang
   e-mail: huangjipengnju@gmail.com
   github: https://github.com/hjptriplebee
   function: improved model can fit XOR
'''''''''''''''''''''''''''''''''''''''''''''''''''''
import tensorflow as tf
import numpy as np

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 1])

w1_1 = tf.Variable(tf.random_normal([2, 1]))
w1_2 = tf.Variable(tf.random_normal([2, 1]))
w2 = tf.Variable(tf.random_normal([2, 1]))

b1_1 = tf.constant(0.1, shape=[1])
b1_2 = tf.constant(0.1, shape=[1])
b2 = tf.constant(0.1, shape=[1])

h1 = tf.nn.relu(tf.matmul(x, w1_1) + b1_1)
h2 = tf.nn.relu(tf.matmul(x, w1_2) + b1_2)

hidden = tf.concat([h1, h2], 1)
out = tf.matmul(hidden, w2) + b2

loss = tf.reduce_mean(tf.square(out - y))

train = tf.train.AdamOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        sess.run(train, feed_dict={x: X, y: Y})
        loss_ = sess.run(loss, feed_dict={x: X, y: Y})
        if not i%200:
            print("step: %d, loss: %.3f"%(i, loss_))
    print("X: %r"%X)
    print("pred: %r"%sess.run(out, feed_dict={x: X}))

step: 0, loss: 0.476
step: 200, loss: 0.034
step: 400, loss: 0.019
step: 600, loss: 0.017
step: 800, loss: 0.015
X: array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])
pred: array([[0.21001229],
       [0.9883025 ],
       [0.9885254 ],
       [0.10054824]], dtype=float32)


In [27]:
#自定义激活函数
from tensorflow.python.framework import ops

def my_stack_def(x, threshold=0.0):
    if x>threshold:
        return 1
    else:
        return 0
my_stack_def = np.vectorize(my_stack_def)
my_stack_def_32 = lambda x: my_stack_def(x).astype(np.float32)


def my_stack_tf(x, name=None):
    with ops.name_scope(name, "my_stack_tf", [x]) as name:
        y = tf.py_func(my_stack_def_32,
                       [x],
                       [tf.float32],
                       name=name,
                       stateful=False)
        return y[0]
    
def my_py_func(func, inp, Tout, stateful=False, name=None, my_grad_func=None):
    # Need to generate a unique name to avoid duplicates:
    random_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    tf.RegisterGradient(random_name)(my_grad_func)  # see _my_relu_grad for grad example
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": random_name, "PyFuncStateless": random_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)
    
def _my_stack_grad(op, pre_grad):
    x = op.inputs[0]
    cur_grad = my_stack_tf(x)
    next_grad = pre_grad * cur_grad
    return next_grad
 
def my_st_tf(x, name=None):
    with ops.name_scope(name, "my_st_tf", [x]) as name:
        y = my_py_func(my_stack_def_32,
                       [x],
                       [tf.float32],
                       stateful=False,
                       name=name,
                       my_grad_func=_my_stack_grad)  # <-- here's the call to the gradient
        return y[0]
    
with tf.Session() as sess:
    x = tf.constant([-0.3, 0.005, 0.08, 0.12])
    y = my_st_tf(x)
    #tf.global_variables_initializer().run()
    print("X: %r"%x)
    print("Y: %r"%y)
    print(x.eval())
    print(y.eval())

X: <tf.Tensor 'Const_50:0' shape=(4,) dtype=float32>
Y: <tf.Tensor 'my_st_tf:0' shape=<unknown> dtype=float32>
[-0.3    0.005  0.08   0.12 ]
[0. 1. 1. 1.]


In [57]:
#给定权重和偏差

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 1])

w1_1 = tf.constant([[-1.],[-1.]])
w1_2 = tf.constant([[1.],[1.]])
w2 = tf.constant([[1.],[1.]])

b1_1 = tf.constant(1.5, shape=[1])
b1_2 = tf.constant(-0.5, shape=[1])
b2 = tf.constant(-1.5, shape=[1])

h1 = my_st_tf(tf.matmul(x, w1_1) + b1_1)
h2 = my_st_tf(tf.matmul(x, w1_2) + b1_2)

hidden = tf.concat([h1, h2], 1)
out = my_st_tf(tf.matmul(hidden, w2) + b2)
loss = tf.reduce_mean(tf.square(out - y))

#train = tf.train.AdamOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
    loss_ = sess.run(loss, feed_dict={x: X, y: Y})
    print("step: %d, loss: %.3f"%(i, loss_))
    #for i in range(1000):
        #for j in range(4):
            #sess.run(train, feed_dict={x: np.expand_dims(X[j], 0), y: np.expand_dims(Y[j], 0)})
        #loss_ = sess.run(loss, feed_dict={x: X, y: Y})
        #print("step: %d, loss: %.3f"%(i, loss_))
    print("X: %r"%X)
    print("pred: %r"%sess.run(out, feed_dict={x: X}))

step: 999, loss: 0.000
X: array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])
pred: array([[0.],
       [1.],
       [1.],
       [0.]], dtype=float32)


In [65]:
#固定转换矩阵，求截距，relu激活

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 1])

w1_1 = tf.constant([[1.],[1.]])
w1_2 = tf.constant([[1.],[1.]])
w2 = tf.constant([[1.],[-2.]])

b1_1 = tf.Variable(tf.random_normal([1, 1]))
b1_2 = tf.Variable(tf.random_normal([1, 1]))
b2 = tf.Variable(tf.random_normal([1, 1]))

h1 = tf.nn.relu(tf.matmul(x, w1_1) + b1_1)
h2 = tf.nn.relu(tf.matmul(x, w1_2) + b1_2)

hidden = tf.concat([h1, h2], 1)
out = tf.matmul(hidden, w2) + b2

loss = tf.reduce_mean(tf.square(out - y))

learning_rate = tf.placeholder(tf.float32)
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        if i < 1000:
          lr = 0.5
        elif i < 3000:
          lr = 0.1
        else:
          lr = 0.01
        sess.run(train, feed_dict={x: X, y: Y, learning_rate:lr})
        loss_ = sess.run(loss, feed_dict={x: X, y: Y})
        if not i % 200:
            print("step: %d, loss: %.3f"%(i, loss_))
    print("X: %r"%X)
    print("pred: %r"%sess.run(out, feed_dict={x: X}))
    print("b1_1: %.3f , b1_2: %.3f , b2: %.3f"%(b1_1.eval(),b1_2.eval(),b2.eval()))

step: 0, loss: 3.348
step: 200, loss: 0.000
step: 400, loss: 0.000
step: 600, loss: 0.000
step: 800, loss: 0.000
step: 1000, loss: 0.000
step: 1200, loss: 0.000
step: 1400, loss: 0.000
step: 1600, loss: 0.000
step: 1800, loss: 0.000
step: 2000, loss: 0.000
step: 2200, loss: 0.000
step: 2400, loss: 0.000
step: 2600, loss: 0.000
step: 2800, loss: 0.000
step: 3000, loss: 0.000
step: 3200, loss: 0.000
step: 3400, loss: 0.000
step: 3600, loss: 0.000
step: 3800, loss: 0.000
step: 4000, loss: 0.000
step: 4200, loss: 0.000
step: 4400, loss: 0.000
step: 4600, loss: 0.000
step: 4800, loss: 0.000
X: array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])
pred: array([[ 0.0000000e+00],
       [ 1.0000000e+00],
       [ 1.0000000e+00],
       [-1.1920929e-07]], dtype=float32)
b1_1: 0.408 , b1_2: -1.000 , b2: -0.408


In [74]:
#固定转换矩阵，求截距，stack激活

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 1])

w1_1 = tf.constant([[1.],[1.]])
w1_2 = tf.constant([[1.],[1.]])
w2 = tf.constant([[1.],[-2.]])

b1_1 = tf.Variable(tf.random_normal([1, 1]))
b1_2 = tf.Variable(tf.random_normal([1, 1]))
b2 = tf.Variable(tf.random_normal([1, 1]))

h1 = my_st_tf(tf.matmul(x, w1_1) + b1_1)
h2 = my_st_tf(tf.matmul(x, w1_2) + b1_2)

hidden = tf.concat([h1, h2], 1)
out = tf.matmul(hidden, w2) + b2
pred = my_st_tf(out)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=out))
learning_rate = tf.placeholder(tf.float32)
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        if i < 1000:
          lr = 0.5
        elif i < 3000:
          lr = 0.1
        else:
          lr = 0.01
        sess.run(train, feed_dict={x: X, y: Y,learning_rate:lr})
        loss_ = sess.run(loss, feed_dict={x: X, y: Y})
        if not i % 500:
            print("step: %d, loss: %.3f"%(i, loss_))
    print("X: %r"%X)
    print("out: %r"%sess.run(out, feed_dict={x: X}))
    print("pred: %r"%sess.run(pred, feed_dict={x: X}))
    print("b1_1: %.3f , b1_2: %.3f , b2: %.3f"%(b1_1.eval(),b1_2.eval(),b2.eval()))

step: 0, loss: 0.731
step: 500, loss: 0.531
step: 1000, loss: 0.532
step: 1500, loss: 0.532
step: 2000, loss: 0.532
step: 2500, loss: 0.533
step: 3000, loss: 0.532
step: 3500, loss: 1.132
step: 4000, loss: 0.532
step: 4500, loss: 1.132
X: array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])
out: array([[ 0.5661192],
       [ 0.5661192],
       [ 0.5661192],
       [-1.4338808]], dtype=float32)
pred: array([[1.],
       [1.],
       [1.],
       [0.]], dtype=float32)
b1_1: 0.082 , b1_2: -1.004 , b2: -0.434


In [68]:
X = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
Y = np.array([[0.], [1.], [1.], [0.]])

x = tf.placeholder(tf.float32, [None, 2])
y = tf.placeholder(tf.float32, [None, 1])

w1 = tf.Variable(tf.random_normal([2, 4]))
w2 = tf.Variable(tf.random_normal([4, 1]))

b1 = tf.Variable(tf.random_normal([4,]))
b2 = tf.Variable(tf.random_normal([1,]))

hidden = tf.nn.relu(tf.matmul(x, w1) + b1)

out = tf.matmul(hidden, w2) + b2

pred = tf.nn.sigmoid(out)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=out))

train = tf.train.AdamOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):       #2000步收敛
        for j in range(4):
            sess.run(train, feed_dict={x: np.expand_dims(X[j], 0), y: np.expand_dims(Y[j], 0)})
        loss_ = sess.run(loss, feed_dict={x: X, y: Y})
        if not i % 500:
            print("step: %d, loss: %.3f"%(i, loss_))
    print("X: %r"%X)
    print("pred: %r"%sess.run(pred, feed_dict={x: X}))
    print("w1:%r"%w1)
    print("w2:%r"%w2)

step: 0, loss: 0.812
step: 500, loss: 0.015
step: 1000, loss: 0.003
step: 1500, loss: 0.001
step: 2000, loss: 0.000
step: 2500, loss: 0.000
step: 3000, loss: 0.000
step: 3500, loss: 0.000
step: 4000, loss: 0.000
step: 4500, loss: 0.000
X: array([[0., 0.],
       [0., 1.],
       [1., 0.],
       [1., 1.]])
pred: array([[1.4375511e-06],
       [1.0000000e+00],
       [9.9999988e-01],
       [1.4375511e-06]], dtype=float32)
w1:<tf.Variable 'Variable_182:0' shape=(2, 4) dtype=float32_ref>
w2:<tf.Variable 'Variable_183:0' shape=(4, 1) dtype=float32_ref>


In [15]:
X.dtype

dtype('float64')